<a href="https://colab.research.google.com/github/BrunoFerCam/BotYugioh/blob/main/BotYugioh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pytelegrambotapi
! pip install requests
! pip install random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 4.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


In [2]:
import telebot
import requests
import random

In [62]:
API_TOKEN = '6754784484:AAEZSpJAWx7px3V2rksmIgtU5-DLzp1CrIk'
bot = telebot.TeleBot(API_TOKEN)

In [63]:
def get_all_cards():
    response = requests.get('https://db.ygoprodeck.com/api/v7/cardinfo.php')
    if response.status_code == 200:
        data = response.json()
        return data.get('data', [])
    return []

# Function to get a random card
def get_random_card():
    cards = get_all_cards()
    if cards:
        card = random.choice(cards)
        return format_card_info(card)
    return "Failed to retrieve card information."

# Function to get a random card based on a specific context
def get_card_by_context(context):
    cards = get_all_cards()
    filtered_cards = []

    # Define keywords and associated card properties
    keywords = {
    'fire': {'attribute': 'FIRE'},
    'water': {'attribute': 'WATER'},
    'earth': {'attribute': 'EARTH'},
    'wind': {'attribute': 'WIND'},
    'light': {'attribute': 'LIGHT'},
    'dark': {'attribute': 'DARK'},
    'quick spell': {'type': 'Spell Card', 'race': 'Quick-Play'},
    'ritual': {'type': 'Ritual Monster'},
    'fusion': {'type': 'Fusion Monster'},
    'synchro': {'type': 'Synchro Monster'},
    'xyz': {'type': 'XYZ Monster'},
    'link': {'type': 'Link Monster'},
    'pendulum': {'type': 'Pendulum Monster'},
    'tuner': {'type': 'Tuner Monster'},
    'field spell': {'type': 'Spell Card', 'race': 'Field'},
    'equip spell': {'type': 'Spell Card', 'race': 'Equip'},
    'quick-play spell': {'type': 'Spell Card', 'race': 'Quick-Play'},
    'continuous spell': {'type': 'Spell Card', 'race': 'Continuous'},
    'normal spell': {'type': 'Spell Card', 'race': 'Normal'},
    'counter trap': {'type': 'Trap Card', 'race': 'Counter'},
    'normal trap': {'type': 'Trap Card', 'race': 'Normal'},
    'continuous trap': {'type': 'Trap Card', 'race': 'Continuous'},
    'counter trap': {'type': 'Trap Card', 'race': 'Counter'},
    'dragon': {'race': 'Dragon'},
    'spellcaster': {'race': 'Spellcaster'},
    'fiend': {'race': 'Fiend'},
    'fairy': {'race': 'Fairy'},
    'beast': {'race': 'Beast'},
    'beast-warrior': {'race': 'Beast-Warrior'},
    'plant': {'race': 'Plant'},
    'insect': {'race': 'Insect'},
    'thunder': {'race': 'Thunder'},
    'aqua': {'race': 'Aqua'},
    'pyro': {'race': 'Pyro'},
    'rock': {'race': 'Rock'},
    'winged-beast': {'race': 'Winged Beast'},
    'machine': {'race': 'Machine'},
    'dinosaur': {'race': 'Dinosaur'},
    'fish': {'race': 'Fish'},
    'sea serpent': {'race': 'Sea Serpent'},
    'reptile': {'race': 'Reptile'},
    'psychic': {'race': 'Psychic'},
    'wyrm': {'race': 'Wyrm'},
    'cyberse': {'race': 'Cyberse'},
    'divine-beast': {'race': 'Divine-Beast'},
    'creator god': {'race': 'Creator God'},
    'rock': {'race': 'Rock'},
    'pyro': {'race': 'Pyro'},
    'sea serpent': {'race': 'Sea Serpent'},
    'spellcaster': {'race': 'Spellcaster'},
    'thunder': {'race': 'Thunder'},
    'warrior': {'race': 'Warrior'},
    'pendulum': {'pendulum_scale': True},
    'normal': {'type': 'Normal'},
    'effect': {'type': 'Effect'},
    'tuner': {'type': 'Tuner'},
    'gemini': {'type': 'Gemini'},
    'union': {'type': 'Union'},
    'toon': {'type': 'Toon'},
    # Marker to recognize Pendulum cards
    # Add more keywords as needed
    }

    context_words = context.split()
    for card in cards:
        match = True
        for word in context_words:
            word_lower = word.lower()
            if word_lower in keywords:
                properties = keywords[word_lower]
                if 'pendulum_scale' in properties:
                    if 'pendulum' not in card['type'].lower():
                        match = False
                        break
                else:
                    if not all(card.get(prop) == value for prop, value in properties.items()):
                        match = False
                        break
        if match:
            filtered_cards.append(card)

    # Return a random card from the filtered list, or a default message if no matches
    if filtered_cards:
        card = random.choice(filtered_cards)
        return format_card_info(card)
    else:
        return "No cards found matching the specified context."

# Function to format card information
def format_card_info(card):
    card_info = (
        f"**Name:** {card['name']}\n"
        f"**Type:** {card['type']}\n"
    )
    if 'Monster' in card['type']:
        card_info += f"**Attribute:** {card['attribute']}\n"
        card_info += f"**ATK:** {card['atk']}\n"
        if 'Link' in card['type']:
            card_info += f"**Link Rating:** {card['linkval']}\n"
            card_info += f"**Link Arrows:** {', '.join(card['linkmarkers'])}\n"
        else:
            card_info += f"**DEF:** {card['def']}\n"
            card_info += f"**Level:** {card['level']}\n"
        if 'Pendulum' in card['type']:
            card_info += f"**Pendulum Scale:** {card['scale']}\n"
        card_info += f"**Race:** {card['race']}\n"
        card_info += f"**Description:** {card['desc']}\n"
    elif 'Spell' in card['type'] or 'Trap' in card['type']:
        card_info += (
            f"**Card Type:** {card['race']}\n"
            f"**Description:** {card['desc']}\n"
        )
    if 'card_images' in card and len(card['card_images']) > 0:
        card_info += f"**Image URL:** {card['card_images'][0]['image_url']}\n"
    return card_info


# Message handler that interprets user's message and provides a card based on context
@bot.message_handler(func=lambda message: message.text is not None)
def interpret_message(message):
    user_message = message.text.lower()
    if "random card" in user_message or "yugioh card" in user_message or "random" in user_message:
        card_info = get_random_card()
    else:
        card_info = get_card_by_context(user_message)
    bot.reply_to(message, card_info, parse_mode='Markdown')



In [ ]:
bot.polling()